<a href="https://colab.research.google.com/github/JMJ-01/JMJ-01.github.io-scientific-programs/blob/main/CHM_615_Assignment_2_Q1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Question: Make a contour plot for the equa􀆟on f(x,y) = (B * (x**4) - C * (x**2) + A * (y**2)) .Iden􀆟fy the internal reac􀆟on coordinate.(Hint: Locate the saddle point)
#ChatGPT was used iteratively to correct my code

import numpy as np
from scipy.optimize import minimize
from ipywidgets import interactive
import matplotlib.pyplot as plt
from matplotlib import cm

# Constants
B = 0.01
C = 5
A = 5

# Define your numerical function
def numerical_function(x):
    # Replace this with your function
    return B * (x[0]**4) - C * (x[0]**2) + A * (x[1]**2)

# Define the gradient of the function
def gradient(vars):
    x, y = vars
    first_partial_x = (4 * B * x**3) + (-2 * C * x)
    first_partial_y = 2 * A * y
    gradient_vector = np.array([first_partial_x, first_partial_y])
    return gradient_vector

# Define the Hessian of the function
def hessian(vars):
    x, y = vars
    second_partial_x_x = 12 * B * x**2 - 2 * C
    second_partial_x_y = 0
    second_partial_y_x = 0
    second_partial_y_y = 2 * A
    hessian_matrix = np.array([[second_partial_x_x, second_partial_x_y],
                               [second_partial_x_y, second_partial_y_y]])
    return hessian_matrix

# Newton's Method for Minimizing Gradient Function
def newtons_method(initial_guess, tolerance=1e-6, max_iterations=100):
    vars = np.array(initial_guess)
    intermediate_points = []  # Store intermediate points

    for i in range(max_iterations):
        intermediate_points.append(vars.copy())  # Store current point
        grad = gradient(vars)
        hess = hessian(vars)
        delta = np.linalg.solve(hess, grad)
        vars = vars - delta
        if np.linalg.norm(delta) < tolerance:
            break

    # Evaluate the function at intermediate points
    function_values = [numerical_function(point) for point in intermediate_points[-10:]]

    # Return optimized variables, last ten intermediate points, and their function values
    return vars, list(zip(intermediate_points[-10:], function_values))

# Initial guess
initial_guess = [3.0, 3.0]

# Perform Newton's Method
result, last_ten_points_with_values = newtons_method(initial_guess)

# Extract the optimized parameters
optimized_x, optimized_y = result

print("Optimized x (saddle point):", optimized_x)
print("Optimized y (saddle point):", optimized_y)

# Check if it's a saddle point (has both positive and negative eigenvalues)
eigenvalues = np.linalg.eigvals(hessian([optimized_x, optimized_y]))
if any(eig > 0 for eig in eigenvalues) and any(eig < 0 for eig in eigenvalues):
    print("Saddle Point found.")
else:
    print("No saddle point found.")

# Print the last ten intermediate points with function values
print("Last ten intermediate points with function values:")
for point, value in last_ten_points_with_values:
    print(f"Point: {point}, Function Value: {value}")

##############################################################################
# PLOTTING SADDLE POINT AND GRAPH
##############################################################################

f = lambda x, y: B * (x**4) - C * (x**2) + A * (y**2)
x = np.linspace(-20, 20, 1000)
y = np.linspace(-20, 20, 1000)
X, Y = np.meshgrid(x, y)
saddle_value = f(optimized_x, optimized_y)
P = f(X, Y)

from mpl_toolkits.mplot3d import Axes3D
from matplotlib.patches import FancyArrowPatch

class Arrow3D(FancyArrowPatch):
    def __init__(self, xs, ys, zs, *args, **kwargs):
        FancyArrowPatch.__init__(self, (0, 0), (0, 0), *args, **kwargs)
        self._verts3d = xs, ys, zs

def plotter(E, A):
    fig = plt.figure(figsize=[12, 8])
    ax = fig.add_subplot(111, projection='3d')
    ax.plot_surface(X, Y, P, cmap=cm.coolwarm)
    ax.scatter(optimized_x, optimized_y, saddle_value, color='red', s=100, label=f'Saddle Point\nValue: {saddle_value:.2f}')

    # Convert last_ten_points and function_values to NumPy arrays
    last_ten_points_array = np.array([point for point, _ in last_ten_points_with_values])
    function_values_array = np.array([value for _, value in last_ten_points_with_values])

    # Plot the last ten intermediate points with their function values
    ax.scatter(last_ten_points_array[:, 0], last_ten_points_array[:, 1], function_values_array+1.5, color='green', s=100, label='Intermediate Points')

    # Plot arrows connecting intermediate points
    for i in range(len(last_ten_points_array) - 1):
        arrow_start = last_ten_points_array[i]
        arrow_end = last_ten_points_array[i + 1]
        ax.quiver(arrow_start[0], arrow_start[1], function_values_array[i],
                  arrow_end[0] - arrow_start[0], arrow_end[1] - arrow_start[1], function_values_array[i + 1] - function_values_array[i],
                  color='blue', arrow_length_ratio=1)

    ax.view_init(elev=E, azim=A)
    plt.legend()
    plt.show()

iplot = interactive(plotter, E=(-90, 90, 5), A=(-90, 90, 5))
iplot